In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math 

In [2]:
code='NIFTY_BANK.csv'
df=pd.read_csv(code)
df.rename(columns={'Date ':'Date','Open ':'Open','High ':'High','Low ':'Low','Close ':'Close'},inplace=True)
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close'], dtype='object')

In [3]:
df.set_index('Date',inplace=True)
df.index=pd.to_datetime(df.index,format="%d/%m/%y")

In [4]:
df.tail()

,Open,High,Low,Close
Date,,,,
2024-02-27,46480.20,46722.25,46324.90,46588.05
2024-02-28,46640.90,46754.55,45852.55,45963.15
2024-02-29,45881.45,46329.65,45661.75,46120.90
2024-03-01,46218.00,47342.25,46218.00,47286.90
2024-03-02,47377.45,47433.85,47237.00,47297.50


In [63]:
class Backtest:
    
    window=1
    entry_price,exit_price,entry_date,exit_date=0,0,0,0
    trade_direction=str()
    
    def __init__(self, df, max_sl,bep_tr):
        self.df=df
        self.max_sl=max_sl
        self.bep_tr=bep_tr
        self.trades=[]
        self.trade_dt=0
        self.initial_investment=100000
        self.max_dd=0
        self.calmer=0
        self.avg_pnl=0
        self.Pivot()
        self.Prev_Pivot_idx()
        self.Prev_Pivot()
        self.execution()
        self.trade_data()
        self.trade_metrics()
        
    def Pivot(self):
        
        pivot=[0 for i in range(len(self.df))]
        
        for i in range(len(self.df)):
            
            if (i - self.window) < 0 or (i+self.window>=len(self.df)):
                pivot[i]=0
            elif self.df.Close.iloc[i] > self.df.Close[i-self.window] and self.df.Close.iloc[i] > self.df.Close[i+self.window] and self.df.Close.iloc[i] < self.df.Close[i-self.window] and self.df.Close.iloc[i] < self.df.Close[i+self.window]:
                pivot[i]=3
            elif self.df.Close.iloc[i] > self.df.Close[i-self.window] and self.df.Close.iloc[i] > self.df.Close[i+self.window]:
                pivot[i]=1
            elif self.df.Close.iloc[i] < self.df.Close[i-self.window] and self.df.Close.iloc[i] < self.df.Close[i+self.window]:
                pivot[i]=2
        
        self.df['isPivot']=pivot 
        
    def Prev_Pivot_idx(self):
        
        self.df = df.reset_index()
        
    
        self.df = self.df.rename_axis('dummy_idx').reset_index()
        self.df['prev_idx'] = self.df.groupby('isPivot')['dummy_idx'].shift()
    
        # convert floats to integers (and NaN to <NA>)
        self.df['prev_idx'] = self.df['prev_idx'].astype('Int64')
        self.df['prev_idx']=self.df['prev_idx'].fillna(0)
        
        self.df.drop(['dummy_idx'],axis=1)
        self.df.set_index('Date',inplace=True)

    def Prev_Pivot(self):
        
        prev_pivot=[0 for i in range(len(self.df))]
        cnt_pvt=0
        
        for i in range(len(self.df)):
            
            if (self.df.isPivot.iloc[i] == 1 or self.df.isPivot.iloc[i] == 2) and cnt_pvt < 2:
                cnt_pvt=cnt_pvt+1
            elif self.df.isPivot.iloc[i] == 1 or self.df.isPivot.iloc[i] == 2:
                idx=self.df.prev_idx.iloc[i]
                prev_pivot[i]=self.df.Close.iloc[idx]        
        
        self.df['Prev_Pivot']=prev_pivot
    
    def trade_log(self):
        
        self.trades.append({'Trade Direction':self.trade_direction,
                        'Entry Date':self.entry_date,
                       'Entry Price':self.entry_price,
                       'Exit Date': self.exit_date,
                      'Exit Price': self.exit_price,
                      'Profit': (self.exit_price - self.entry_price)*15 if self.trade_direction=='Long' else (self.entry_price - self.exit_price)*15})  
       
    def trade_data(self):
        
        self.trade_dt=pd.DataFrame(self.trades)
    
    def trade_metrics(self):
        
        mean_pnl=0
        
        self.trade_dt['Cum_Pnl']=self.trade_dt['Profit'].cumsum()
        self.trade_dt['Equity']=self.trade_dt['Cum_Pnl']+self.initial_investment
        self.trade_dt['Peak']=self.trade_dt['Equity'].cummax()
        self.trade_dt['Drawdown']=((self.trade_dt['Equity']-self.trade_dt['Peak'])/self.trade_dt['Peak'])*100
        #self.trade_dt['Drawdown']=self.trade_dt['Equity']-self.trade_dt['Peak']
        self.max_dd=self.trade_dt['Drawdown'].min()
        mean_pnl=self.trade_dt.Cum_Pnl.iloc[-1]/2
        self.avg_pnl=(mean_pnl/self.initial_investment)*100
        self.calmer=abs(self.avg_pnl/self.max_dd)
        
    @staticmethod
    def long_stop(long_price,system_low,sl):
        system_points=long_price - system_low
    
        if  long_price*(sl/100) < system_points:
            stop_price=round(long_price*(1-(sl/100)),2)
        else:
            stop_price=system_low
    
        return stop_price
    
    @staticmethod
    def short_stop(short_price,system_high,sl):
        system_points=system_high - short_price
    
        if short_price*(sl/100) < system_points:
            stop_price=round(short_price*(1+(sl/100)),2)
        else:
            stop_price=system_high 
    
        return stop_price
    
    def execution(self):
        
        swing_high,swing_low=0,0
        long_price,short_price=0,0
        st,gap_up,gap_down=0,0,0
        pos_sl,bep=0,0
        
        for i in range(len(self.df)):
            
            if self.df.isPivot.iloc[i]==1:
                if self.df.Prev_Pivot.iloc[i]==0 or self.df.Prev_Pivot.iloc[i] > self.df.Close.iloc[i] or st==1:
                    swing_high=self.df.Close.iloc[i]
                else:
                    swing_high=self.df.Prev_Pivot.iloc[i]
                gap_up=0
            elif self.df.isPivot.iloc[i]==2:
                if self.df.Prev_Pivot.iloc[i]==0 or self.df.Prev_Pivot.iloc[i] < self.df.Close.iloc[i] or st==2:
                    swing_low=self.df.Close.iloc[i]
                else:
                    swing_low=self.df.Prev_Pivot.iloc[i]
                gap_down=0
            
            if st==1:
                if self.df.Open.iloc[i] < swing_low*(1-(self.max_sl/100)):
                    gap_down=1
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Open.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Open.iloc[i] < self.df.Close.iloc[i-1]*(1-(self.max_sl/100)):
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif pos_sl!=0 and self.df.Close.iloc[i] < pos_sl:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Close.iloc[i] > long_price*(1+(self.bep_tr/100)) and bep==0:
                    pos_sl=self.entry_price
                    bep=bep+1
                
            if st==2:
                if self.df.Open.iloc[i] > swing_high*(1+(self.max_sl/100)):
                    gap_up=1
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Open.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Open.iloc[i] > self.df.Close.iloc[i-1]*(1+(self.max_sl/100)):
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif pos_sl!=0 and self.df.Close.iloc[i] > pos_sl:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    pos_sl,bep,st=0,0,0
                elif self.df.Close.iloc[i] < short_price*(1-(self.bep_tr/100)) and bep==0:
                    pos_sl=self.entry_price
                    bep=bep+1
                
            
            if swing_high!=0 and self.df.Close.iloc[i] > swing_high and gap_up==0 and (st==0 or st==2):
                long_price=swing_high
                pos_sl,bep=0,0
                if st==2:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Long'
                else:
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Long'
                st=1
                if Backtest.long_stop(long_price,swing_low,self.max_sl) != swing_low:
                    pos_sl=Backtest.long_stop(long_price,swing_low,self.max_sl)
                    #print(pos_sl)
                    #print(self.df.index[i])
                else:
                    #print(pos_sl)
                    #print(self.df.index[i])
                    pos_sl=0
                
            elif swing_low!=0 and self.df.Close.iloc[i] < swing_low and gap_down==0 and (st==0 or st==1):
                short_price=swing_low
                pos_sl,bep=0,0
                if st==1:
                    self.exit_date=self.df.index[i]
                    self.exit_price=self.df.Close.iloc[i]
                    self.trade_log()
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Short'
                else:
                    self.entry_date=self.df.index[i]
                    self.entry_price=self.df.Close.iloc[i]
                    self.trade_direction='Short'
                st=2
                if Backtest.short_stop(short_price,swing_high,self.max_sl) != swing_high:
                    pos_sl=Backtest.short_stop(short_price,swing_high,self.max_sl)
                else:
                    pos_sl=0
                
             

In [78]:
instance=Backtest(df,1.3,1.7)

In [79]:
instance.df.head(20)

,dummy_idx,Open,High,Low,Close,isPivot,prev_idx,Prev_Pivot
Date,,,,,,,,
2022-01-03,0,35585.20,36492.10,35526.60,36421.90,0,0,0.00
2022-01-04,1,36551.25,36887.80,36374.40,36840.15,0,0,0.00
2022-01-05,2,36943.55,37862.40,36756.35,37695.90,1,0,0.00
2022-01-06,3,37242.55,37752.50,37058.45,37490.25,2,0,0.00
2022-01-07,4,37667.05,38134.85,37427.80,37739.60,0,1,0.00
2022-01-10,5,37930.55,38400.35,37929.35,38347.90,0,4,0.00
2022-01-11,6,38370.00,38504.60,38031.75,38442.20,0,5,0.00
2022-01-12,7,38719.60,38851.45,38604.60,38727.55,1,2,37695.90
2022-01-13,8,38717.55,38717.55,38376.15,38469.95,0,6,0.00


In [80]:
instance.trade_dt

,Trade Direction,Entry Date,Entry Price,Exit Date,Exit Price,Profit,Cum_Pnl,Equity,Peak,Drawdown
0,Long,2022-01-07,37739.60,2022-01-21,37574.30,-2479.50,-2479.50,97520.50,97520.5,0.000000
1,Short,2022-01-24,36947.55,2022-01-27,37982.10,-15518.25,-17997.75,82002.25,97520.5,-15.912808
2,Long,2022-02-01,38505.50,2022-02-07,37995.45,-7650.75,-25648.50,74351.50,97520.5,-23.758082
3,Short,2022-02-14,36908.55,2022-02-25,36430.75,7167.00,-18481.50,81518.50,97520.5,-16.408858
4,Short,2022-03-03,34944.30,2022-03-10,34475.60,7030.50,-11451.00,88549.00,97520.5,-9.199604
...,...,...,...,...,...,...,...,...,...,...
61,Short,2024-01-08,47450.25,2024-01-29,45442.35,30118.50,259210.50,359210.50,359210.5,0.000000
62,Long,2024-01-29,45442.35,2024-02-08,45012.00,-6455.25,252755.25,352755.25,359210.5,-1.797066
63,Short,2024-02-08,45012.00,2024-02-13,45502.40,-7356.00,245399.25,345399.25,359210.5,-3.844890
64,Long,2024-02-14,45908.30,2024-02-28,45963.15,822.75,246222.00,346222.00,359210.5,-3.615846


In [81]:
instance.max_dd

-25.621535984741534

In [82]:
instance.avg_pnl

113.18287499999997

In [83]:
instance.calmer

4.417489843989216

In [21]:
# input the number of rows
N = 3
# input the number of columns
M = 3
# initializing the matrix
res = [ [ i*j for i in range(N) ] for j in range(M) ]

In [22]:
print(res)

[[0, 0, 0], [0, 1, 2], [0, 2, 4]]


In [26]:
max_sl=21
tr_sl=21
d = pd.DataFrame(index=range(tr_sl),columns=range(max_sl))

In [34]:
max_cal=0
op_sl=0
op_tr=0
cnt=0
instances=[]
#val1 = max_sl
#val2 = bep_tr

for i in range(10,31):
    for j in range(10,31):
        val1=round(i*0.1,1)
        val2=round(j*0.1,1)
        cal=Backtest(df,val1,val2).calmer
        if cal > max_cal:
        #print(val1,val2)
#print(cnt)
    
    


1.0 1.0
1.0 1.1
1.0 1.2
1.0 1.3
1.0 1.4
1.0 1.5
1.0 1.6
1.0 1.7
1.0 1.8
1.0 1.9
1.0 2.0
1.0 2.1
1.0 2.2
1.0 2.3
1.0 2.4
1.0 2.5
1.0 2.6
1.0 2.7
1.0 2.8
1.0 2.9
1.0 3.0
1.1 1.0
1.1 1.1
1.1 1.2
1.1 1.3
1.1 1.4
1.1 1.5
1.1 1.6
1.1 1.7
1.1 1.8
1.1 1.9
1.1 2.0
1.1 2.1
1.1 2.2
1.1 2.3
1.1 2.4
1.1 2.5
1.1 2.6
1.1 2.7
1.1 2.8
1.1 2.9
1.1 3.0
1.2 1.0
1.2 1.1
1.2 1.2
1.2 1.3
1.2 1.4
1.2 1.5
1.2 1.6
1.2 1.7
1.2 1.8
1.2 1.9
1.2 2.0
1.2 2.1
1.2 2.2
1.2 2.3
1.2 2.4
1.2 2.5
1.2 2.6
1.2 2.7
1.2 2.8
1.2 2.9
1.2 3.0
1.3 1.0
1.3 1.1
1.3 1.2
1.3 1.3
1.3 1.4
1.3 1.5
1.3 1.6
1.3 1.7
1.3 1.8
1.3 1.9
1.3 2.0
1.3 2.1
1.3 2.2
1.3 2.3
1.3 2.4
1.3 2.5
1.3 2.6
1.3 2.7
1.3 2.8
1.3 2.9
1.3 3.0
1.4 1.0
1.4 1.1
1.4 1.2
1.4 1.3
1.4 1.4
1.4 1.5
1.4 1.6
1.4 1.7
1.4 1.8
1.4 1.9
1.4 2.0
1.4 2.1
1.4 2.2
1.4 2.3
1.4 2.4
1.4 2.5
1.4 2.6
1.4 2.7
1.4 2.8
1.4 2.9
1.4 3.0
1.5 1.0
1.5 1.1
1.5 1.2
1.5 1.3
1.5 1.4
1.5 1.5
1.5 1.6
1.5 1.7
1.5 1.8
1.5 1.9
1.5 2.0
1.5 2.1
1.5 2.2
1.5 2.3
1.5 2.4
1.5 2.5
1.5 2.6
1.5 2.7
1.5 2.8
1.5 2.9


In [73]:
rows = 20
cols = 20

# Initialize the matrix
instances = []

# Generate the matrix
for i in range(10,31):
    row = []
    for j in range(10,31):
        # Here, you can save whatever objects you want in the matrix
        # For demonstration, let's save integers from 1 to 441
        op_sl = round(i*0.1,1) 
        op_tr= round(j*0.1,1)
        row.append(Backtest(df,op_sl,op_tr).calmer)
    instances.append(row)

In [74]:
dk=pd.DataFrame(instances)

In [75]:
dk.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,3.225524,3.366261,3.313404,2.224468,2.106135,2.106135,2.106135,4.096609,3.975695,3.975695,...,3.975695,3.975695,3.975695,3.975695,3.975695,3.975695,3.975695,3.423722,3.423722,3.423722
1,3.309390,3.450128,3.397271,2.282491,2.164158,2.164158,2.164158,3.943967,3.825151,3.825151,...,3.825151,3.825151,3.825151,3.825151,3.825151,3.825151,3.825151,3.290619,3.290619,3.290619
2,3.335797,3.476535,3.423678,2.300761,2.182428,2.182428,2.182428,3.979401,3.860585,3.860585,...,3.860585,3.860585,3.860585,3.860585,3.860585,3.860585,3.860585,3.322637,3.322637,3.322637
3,3.662276,3.803014,3.750157,2.526636,2.408303,2.408303,2.408303,4.417490,4.298674,4.298674,...,4.298674,4.298674,4.298674,4.298674,4.298674,4.298674,4.298674,3.718492,3.718492,3.718492
4,2.883591,2.997626,2.954798,2.112777,2.010942,2.010942,2.010942,3.151202,3.060793,3.060793,...,3.060793,3.060793,3.060793,3.060793,3.060793,3.060793,3.060793,2.639582,2.639582,2.639582


In [77]:
dk.to_csv('optimiser.csv')